# This Notebook performs retrivel queries for one set of image features.

In [41]:
# Import section
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from skimage.filters import gabor_kernel
from scipy import signal as sg
from collections import OrderedDict
import math

In [40]:
# Feature type.
# feature_type = 'mean_stdev'
# feature_type = 'Gabor_2_3'
feature_type = 'Gabor_3_4'

cwd=os.getcwd()

# The filename with the features to use (including the folder).
feature_filename = cwd+'/features/'+feature_type + '.txt'
#print("Feature filename: " + feature_filename)

# Read the features.
features = np.genfromtxt(feature_filename, delimiter=',')

fdim = np.shape(features)[1]
print(feature_type + ' features have dimension:' + str(fdim))

Gabor_3_4 features have dimension:24


In [29]:
# Number of images.
n_images = 218

# Read image names and classes .csv file.
# The .csv file containing the image names and classes.
image_file = cwd + '/image_names_classes.csv'
image_names_classes = pd.read_csv(image_file, header=None )

In [30]:
# Dictionary to store precision and recall for the queries performed.
precision_recall = OrderedDict()

In [31]:
# Dictionary to store the class and class size of birds.
bird_class_size_dict = OrderedDict()

# Where the ROI .csv files are located.
rois_dir = cwd + '/ROIs/'

# The .csv file with the info about the ROIs.
rois_info_file = cwd+'/ROIs/birds.csv'
rois_info = pd.read_csv(rois_info_file)

# Store the class and class size of birds.
for i in range(len(list(rois_info['bird']))):
    bird_class_size_dict[i+1]=rois_info['roi_count'].iloc[i]

# Number of images.
n_images = 218

In [32]:
def L2_dist(feature1, feature2):
    # Get dimension.
    dim = np.shape(feature1)[0]

    result = 0
    for k in range(dim):
        result = result + ((feature1[k] - feature2[k])**2)
    result = result ** 0.5

    return(result)

def L1_dist(feature1, feature2):
    # Get dimension.
    dim = np.shape(feature1)[0]

    result = 0
    for k in range(dim):
        result = result + abs(feature1[k] - feature2[k])

    return(result)

In [33]:
# Perform the retrievals using each image as a query.
for query_image in range(n_images):
    # Compute distance between query feature vector and every target image's feature vector (including the query).
    distances = np.zeros(n_images)
            
    for j in range(n_images):
        # Can choose difference distance measures;
#        distances[j] = L1_dist(features[query_image], features[j])
        distances[j] = L2_dist(features[query_image], features[j])
#        print('distances['+str(j)+'] = '+str(distances[j]))
                
    # Get the indices of the sorted distances.
    sorted_index = np.argsort(distances)
    
    # Get the class and class size of query image
    class_qimg=image_names_classes[1][query_image]
    class_qimg_size= bird_class_size_dict[class_qimg]

    temp_prec_arr=[]
    temp_recal_arr=[]

    # Compute precision and recall for different number of retrieved images.
    for k in range(1,n_images+1):
        prec_num=0
        prec_denom=k
        recal_denom=class_qimg_size
        recal_num=0
        for z in sorted_index[0:k]:
            if class_qimg==image_names_classes[1][z]:
                 prec_num+=1
                 recal_num+=1
        temp_prec_arr.append(prec_num/prec_denom)
        temp_recal_arr.append(recal_num/recal_denom)

        precision_recall[query_image]=[temp_prec_arr,temp_recal_arr]

### Interpolate precision for fixed recall values.

In [34]:
pre_defined_recall = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]

In [35]:
# Empty dictionary to hold the interpolated values.
precision_recall_inter = OrderedDict()

In [36]:
#Function to return the interpolated precision value

def interpolate_p_r(p_arr,r_arr):
    temp_p=[]
    for i in pre_defined_recall: 
        for j in range(len(r_arr)):
            if r_arr[j]>=i:
                temp_p.append(p_arr[j])
                break
    return temp_p    

In [37]:
# Interpolate precision values at fixed recall values.
for k,v in precision_recall.items():
    ans = interpolate_p_r(v[0],v[1])
    precision_recall_inter[k] = ans

In [38]:
# Average the interpolated precision values over all queries.
average_prec = np.zeros(len(pre_defined_recall))
for k,v in precision_recall_inter.items():
    average_prec=np.add(average_prec,v)
for i in range(len(average_prec)):
    average_prec[i]=average_prec[i]/n_images

print(average_prec)


[1.         0.90048965 0.79901856 0.74876807 0.70807945 0.64719684
 0.61850895 0.56327892 0.47290732 0.39819197 0.32843474]


In [39]:
# Create precision_recall folder if it doesn't exist.
try:
    os.makedirs(cwd+'/precision_recall')
except:
    pass

# Save the average precision values as a .csv file.
average_precision_filename = cwd+'/precision_recall/' + feature_type + '_p_r.txt'

np.savetxt(average_precision_filename, average_prec, delimiter=',')